In [1]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install pdfplumber --no-cache-dir

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------- ------------------ 6.8/12.9 MB 35.3 MB/s eta 0:00:01
     ---------------------------------------- 12.9/12.9 MB 32.4 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 5.6/5.6 MB 41.9 MB/s  0:00:00
   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ---------------------------------------- 3.4/3.4 MB 49.4 MB/s  0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 51.1 MB/s  0:00:00

   ------ --------------------------------- 1/6 [pycparser]
   ------------- -------------------------- 2/6 [cffi]
   -------------------- ------------------- 3/6 [cryptography]
   -------------------------- ------------- 4/6 [pdfminer.si

In [2]:
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode

In [3]:
nlp = spacy.load("es_core_news_sm")  #cargar el modelo en español

In [6]:
pdf_path = "C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_2/certificado.pdf"
# Extraer texto del PDF
with pdfplumber.open(pdf_path) as pdf:
    texto_pdf = ""
    for pagina in pdf.pages:
        texto_pdf += pagina.extract_text()

In [7]:
header_pattern = r"Cámara de Comercio de Barranquilla\s*CERTIFICADO DE EXISTENCIA Y REPRESENTACION LEGAL O\s*DE INSCRIPCION DE DOCUMENTOS\.\s*Fecha de expedición:.*?\nRecibo No\..*?\nCODIGO DE VERIFICACIÓN:.*?\n"

In [8]:
def extract_text_without_header(pdf_path):
    extracted_text = []  # Lista donde almacenaremos el texto de cada página

    with pdfplumber.open(pdf_path) as pdf:  # Abrir el archivo PDF
        for page in pdf.pages:  # Iterar por cada página del PDF
            text = page.extract_text()  # Extraer texto de la página
            if text:  # Verificar si hay texto en la página
                text = re.sub(header_pattern, "", text, flags=re.DOTALL)  # Eliminar encabezado con regex
                extracted_text.append(text)  # Guardar el texto limpio en la lista

    return " ".join(extracted_text)  # Unir todas las páginas en un solo texto

In [9]:
def preprocess_text(text):
    text = unicodedata.normalize("NFKD", text)  # Normalizar caracteres Unicode (ej. á → a)

    doc = nlp(text.lower())  # Convertir todo el texto a minúsculas y procesarlo con spaCy

    tokens = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return tokens

In [10]:
raw_text = extract_text_without_header(pdf_path)  # Extraer texto limpio del PDF
tokens = preprocess_text(raw_text)  # Preprocesar el texto con spaCy

print(tokens[:50])  # Mostrar los primeros 50 tokens como prueba

['verifique', 'contenido', 'confiabilidad', 'certificado', 'ingresando', 'digite', 'visualice', 'imagen', 'generada', 'momento', 'ilimitada', 'calendario', 'contados', 'fecha', 'atencion', 'comerciante', 'cumplido', 'deber', 'legal', 'renovar', 'matricula', 'mercantil', 'd', 's', 'matricula', 'mercantil', 'proporciona', 'seguridad', 'confianza', 'ean', 'n', 'renueve', 'matricula', 'mercantil', 'tardar', 'marzo', 'v', 'i', 'c', 'n', 'n', 's', 'fundamento', 'inscrripciones', 'efectu', 'adas', 'registro', 'mercantil', 'comercio', 'certiefica']


In [12]:
with open("C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_2/certificado.pdf", "w", encoding="utf-8") as f:
    f.write(" ".join(tokens))

In [13]:
texto_sin_encabezados = re.sub(header_pattern, "", texto_pdf, flags=re.DOTALL)
print(texto_sin_encabezados[:500])  # Mostrar los primeros 500 caracteres como prueba

-------------------------------------------------------------------------------
Verifique el contenido y confiabilidad de este certificado, ingresando a
www.camarabaq.org.co/ y digite el código, para que visualice la imagen generada
al momento de su expedición. La verificación se puede realizar de manera
ilimitada, durante 60 días calendario contados a partir de la fecha de su
expedición.
--------------------------------------------------------------------------------
***************************
